# 간단한 음성인식기

- 참고 블로그 : https://opac.tistory.com/4
- 참고 : https://wikidocs.net/book/8056

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     3.0.0
autograd                      1.4
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.8

In [ ]:
# 신경망 언어모델(Neural Language Models) 기반 딥러닝 라이브러리 : transformers
# !pip install transformers

# mp3 파일 및 wav 파일 재생
# !pip install playsound

# mp3 파일 wav 변환할때 필요
# pip install pydub
# !apt-get install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
# 음성처리 라이브러리

# pytorch 딥러닝 프레임워크, 페이스북 AI팀에 의해 개발
# 데이터 작업과 모델 생성, 최적화 등 .. 간편하게 가능
import torch
# 파이썬 음성데이터 분석해주는 라이브러리
import librosa
# 응 뭔지 알아 ㅋㅋㅋㅋ 아는게 나와서 기분이 좋군
import numpy as np
# 음성데이터를 가져올 때 사용
import soundfile as sf
# wav 파일 가져올 때 (요즘도 wav 파일을 쓰는가..?)
from scipy.io import wavfile
# 음성데이터 재생할 때 필요
from IPython.display import Audio
# Wav2Vec 프랜스포터 라이브러리, 음성데이터를 백터값으로 변환 , 음성인식에 매우 좋은 성능
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [ ]:
# # mp3 파일 및 wav 파일 재생
# # playsound.playsound('파일경로')
# import playsound

In [ ]:
# wav 파일
file_name = ('구글경로/notebook.wav')
# Audio : 앞에 대문자여야함 .. 
Audio(file_name)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 이전에 학습된 데이터를 이용함.
# pre-trained 모델

# 토크나이저 : 입력된 텍스트를 모델에서 처리할 수 있는 데이터로 변환.
# 모델은 숫자만 처리할 수 있음. 
# 텍스트 입력 → 숫자 데이터로 변환.
# 모델을 로드하고 저장 : from_pretrained() / save_pretrained()
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

# 이미 학습된 모델을 가져옴.
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# wav 파일
data = wavfile.read(file_name)

frame_rate = data[0]
sound_data = data[1]
print(data)
print('Sampling rate :', frame_rate,'Hz')

(48000, array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int16))
Sampling rate : 48000 Hz


In [ ]:
# 데이터를 시계열로

# librosa 라이브러리
input_audio, _ = librosa.load(file_name, sr=16000)
input_audio, _ 

(array([0., 0., 0., ..., 0., 0., 0.], dtype=float32), 16000)

In [ ]:
# 음성파일 토크나이저
input_values = tokenizer(input_audio, return_tensors="pt").input_values

# 확률이 아닌 모델의 정규화되지 않은 원시점수
# 트렌스포머 모델은 logits값을 출력
# 일반적으로 학습을 위한 손실함수(loss function)는 최종 활성화함수(activation function, e.g., SoftMax)와
# 실제 손실함수(actual loss function, e.g., cross entropy)를 모두 구현되기 때문.
logits = model(input_values).logits

# pytorch라이브러리 argmax
predicted_ids = torch.argmax(logits, dim=-1)

# 텍스트 변환하기 위해 
# tokenizer.batch_decode()
transcription = tokenizer.batch_decode(predicted_ids)[0]

In [ ]:
# 결과
transcription

"I HAVE NO ONE SPECIAL JUST A COMMON MAN WITH COMMON THOUGHTS I'VE LED A COMMON LIFE THERE ARE NO MONUMENTS DEDICATED TO ME AND MY NAME WILL SOON BE FORGOTTEN BUT IN ONE RESPECT I'VE SUCCEEDED AS GLORIOUSLY AS ANYONE WHO EVER LIVED I'VE LOVED ANOTHER WITH ALL MY HEART AND SOUL AND FOR ME THAT HAS ALWAYS BEEN ENOUGH"



---



In [ ]:
# mp3 파일
file_name2 = ('구글경로/내가 잘못했네 (192 kbps).mp3')
Audio(file_name2)

In [ ]:
# mp3 파일을 wav 파일로 변환
# pydub 모듈 이용
# from_mp3() : mp3 파일 읽고 / export() : 파일 내보내기
# 이렇게 하는 이유? 
# mp3 불러오는거 모르겠음ㅋㅋㅋ

In [ ]:
from pydub import AudioSegment
import scipy.io.wavfile

# 파일
mp3_file = ('구글경로/내가 잘못했네 (192 kbps).mp3')
wav_file = ('구글경로/내가 잘못했네 (192 kbps).wav')

# 변환
audseg = AudioSegment.from_mp3('구글경로/내가 잘못했네 (192 kbps).mp3')
audseg.export(wav_file, format='wav')

<_io.BufferedRandom name='/content/drive/MyDrive/부산대 AI/프로젝트_파이널/test/내가 잘못했네 (192 kbps).wav'>

In [ ]:
# wav 파일
file_name3 = ('구글경로/내가 잘못했네 (192 kbps).wav')
Audio(file_name3)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
tokenizer2 = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

data2 = wavfile.read(file_name3)

frame_rate2 = data2[0]
sound_data2 = data2[1]
print(data2)
print('Sampling rate :', frame_rate2,'Hz')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:757: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(48000, array([[0, 0],
       [0, 0],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int16))
Sampling rate : 48000 Hz


In [ ]:
input_audio2, _ = librosa.load(file_name3, sr=16000)
input_audio2, _ 

(array([-5.4264834e-08,  2.4935019e-08,  1.7925370e-08, ...,
        -6.2023428e-06, -6.3208854e-06,  1.8060448e-06], dtype=float32), 16000)

In [ ]:
input_values2 = tokenizer2(input_audio2, return_tensors="pt").input_values
logits2 = model2(input_values2).logits

In [ ]:
predicted_ids2 = torch.argmax(logits2, dim=-1)
transcription2 = tokenizer2.batch_decode(predicted_ids2)[0]

In [ ]:
# 결과
transcription2

'CONSA DE CRUCMANDEGE A THE AS AMILTANE ARE THEY ARE CICES  RIPENES I WULLE WULL A CUSA O MA TEDANAS LAIR AND TA TELL ME WHO CAN CE COMISAD MAN CONS A A TO PUCAG APONESS OF THE MAN THE AT A PIAK LIK ONACO MN THER A DA WITH TA SLEVER OR COO TA ON UCA HIS LAT'

### 한국어 안되는 이유?

- 사전 학습된 모델을 이용해서 테스트 함.
- 사전 학습된 모델이 한국어로 한게 아니기 때문.
